# ADA / Applied Data Analysis
<h2 style="color:#a8a8a8">Homework 3 - Visualization<br>
Aimée Montero, Alfonso Peterssen, Cyriaque Brousse</h2>

In this homework we will practice with interactive visualization, which is the key ingredient of many successful viz (especially when it comes to infographics).
You will be working with the P3 database of the [SNSF](http://www.snf.ch/en/Pages/default.aspx) (Swiss National Science Foundation).
As you can see from their [entry page](http://p3.snf.ch/), P3 already offers some ready-made viz, but we want to build a more advanced one for the sake
of quick data exploration. Therefore, start by [downloading the raw data](http://p3.snf.ch/Pages/DataAndDocumentation.aspx) (just for the Grant Export), and read carefully
the documentation to understand the schema.

Build a [Choropleth map](https://en.wikipedia.org/wiki/Choropleth_map) which shows intuitively (i.e., use colors wisely) how much grant money goes to each Swiss canton.
To do so, you will need to use the provided TopoJSON file, combined with the Choropleth map example you can find in the Folium README file.

<ol class="toc-item"><li><a href="#Part-1---Data-import">Part 1 - Data import</a><ol class="toc-item"><li><a href="#1a---Indexing">1a - Indexing</a></li><li><a href="#1b---Cleaning-the-data">1b - Cleaning the data</a></li></ol></li><li><a href="#Part-2---Mapping-universities-to-cantons">Part 2 - Mapping universities to cantons</a><ol class="toc-item"><li><a href="#2a---Defining-abstractions">2a - Defining abstractions</a></li><li><a href="#2b---Using-caching">2b - Using caching</a></li><li><a href="#2c---Retrieving-the-canton-for-each-entry">2c - Retrieving the canton for each entry</a></li><li><a href="#2d---Putting-everything-back-together">2d - Putting everything back together</a></li></ol></li><li><a href="#Part-3---Visualization">Part 3 - Visualization</a><ol class="toc-item"><li><a href="#3a---Why-are-you-not-Swiss??">3a - Why are you not Swiss??</a></li><li><a href="#3b---Summing-grants-per-canton">3b - Summing grants per canton</a></li><li><a href="#3c---Final-map">3c - Final map</a></li></ol></li></ol>

## Part 1 - Data import

Let's import the required libraries:

In [1]:
import folium
import pandas as pd
import numpy as np
import requests
import json

<p>In this homework, we tested the efficiency of the suggested library and that of the Google Places API. The latter has a much higher retrieval rate of the location for a given university, so we will use it.</p>

<p>We define the constants for the Google APIs:</p>

In [2]:
# change this flag to force overwrite of the saved values by fresh data from Google APIs (takes a long time)
force_download = False

In [3]:
places_api_url = 'https://maps.googleapis.com/maps/api/place/textsearch/json'
geocode_api_url = 'http://maps.googleapis.com/maps/api/geocode/json'

Read the SNF data from the provided CSV file:

In [4]:
data = pd.read_csv('data/grantexport.csv', sep=';')

We keep only the columns that are relevant to us and rename then accordingly:

In [8]:
cols = {'Project Number' : 'pnr',
       'Project Title' : 'title',
       'University' : 'univ',
       'Institution' : 'inst',
       'Approved Amount' : 'amount'}
data = data.rename(columns=cols)
data = data[[v for (k,v) in cols.items()]]

***Size of raw data (number of entries)***

In [9]:
len(data)

63969

### 1a - Indexing

According to the documentation, the values in the field `pnr` are unique. We check this and use it as an index for our data frame.

In [10]:
data.pnr.is_unique

True

In [11]:
data = data.set_index('pnr')

<p>We discard the values in the `amount` field that are not numeric. The documentation states that *"This amount is not indicated in the case of mobility fellowships since it depends on administrative factors, typically the destination, cost of living, family allowances (if applicable) and exchange rate differences."*.</p>

<p>The `coerce` argument will change all the `amount` values that are not actual number into `NaN`.</p>

In [12]:
data.amount = pd.to_numeric(data.amount, errors='coerce')
data.sample(5)

,univ,amount,title,inst
pnr,,,,
102913,Université de Lausanne - LA,1369147.0,"Evolution, maintenance and signalling function...",Institute of Ecology University of Lausanne Bâ...
136558,Eidg. Material und Prüfungsanstalt - EMPA,431944.0,Modified Nanofibrillated Cellulose in Wood Coa...,Empa Wood Laboratory
161760,NaN,NaN,La surmortalité des jeunes adultes : causes et...,Institut national d'études démographiques INED
29255,NaN,330.0,Exil der deutschen Arbeiterbewegung in der Sch...,UNI: Berlin D-O
67740,EPF Lausanne - EPFL,173604.0,Intelligent computational control for complex ...,Institut de structures EPFL - IS - ENAC


### 1b - Cleaning the data

The number of rows for which the `amount` field is not a number is:

In [13]:
data.amount.isnull().sum()

10910

We notice that some entries have fields that are not specified. We use the following replacement policy, which is important to keep in mind for later:

In [15]:
data = data.replace('Nicht zuteilbar - NA', np.NaN)
data = data.replace('NPO (Biblioth., Museen, Verwalt.) - NPO', 'NPO')
data = data.replace('Firmen/Privatwirtschaft - FP', 'FP')
data = data.replace('Weitere Institute - FINST', 'FINST')
data = data.dropna()
data.sample(5)

,univ,amount,title,inst
pnr,,,,
27082,EPF Lausanne - EPFL,333038.0,Colloidal processing of anisotropic particles,Institut des matériaux EPFL - STI - IMX
107956,EPF Lausanne - EPFL,206936.0,Excited state geometries and photochemical rea...,Laboratoire de spectroscopie ultrarapide EPFL ...
131828,EPF Lausanne - EPFL,112363.0,Static and dynamic force spectroscopy in virol...,Laboratoire de physique de la matière vivante ...
142213,Universität Zürich - ZH,362821.0,Effects of Sex Steroids on Adult Stem/Progenit...,Klinik für Reproduktions-Endokrinologie Depart...
57477,Université de Fribourg - FR,3250.0,Juliusz Slowacki (1809-1849). Probleme einer P...,Slavisches Seminar Universität Freiburg


The ***number of entries*** we will be working with is then:

In [16]:
data_size = len(data)
data_size

47048

## Part 2 - Mapping universities to cantons

<p>We want to display, somehow, the amount of money granted to each canton.<br>
In technical terms, it means we have to find a mapping for  each `univ` value to its corresponding `canton` value.<p>

<p>To do that, we will use the Google Places API. We take care not to divulge the API key, which will be placed in a separate file, ignored by Git.<br>
The API call will return a location for each university. We then input this location in the Reverse Geocoding API, which will return a canton.</p>

### 2a - Defining abstractions

First, we need to import the Google API key from the key file:

In [17]:
api_key = !head api_key
api_key = api_key[0]

We define the following helper method for fetching JSON from Google APIs:

In [18]:
def fetch_json(url, params):
    ''' Fetches the json object resulting from the query of the url with
        the given params.
        Checks that the status is OK, otherwise returns None.
        If the status is OK, then it will return the first result,
        if it exists, None otherwise.
    '''
    response = requests.get(url, params)
    obj = json.loads(response.text)
    
    if obj['status'] != 'OK':
        #print('[E] status was', obj['status'])
        return None
    
    if len(obj['results']) < 1:
        return None
    
    return obj['results'][0]

We define the following functions to get the canton from the university name, or the institution name if the university name is a non-profit or a private organization:

In [20]:
def get_query(univ, inst):
    ''' Returns the university if it is not a non-profit or private organization, the institution otherwise.
    '''
    return univ if univ not in ('NPO', 'FP', 'FINST') else inst

It is here that the "magic" happens:<br>
for a pair `(university, institution)`, this function will figure out which of the two members of the tuple to feed to the Google APIs, and from the response, compute the canton for this pair.<br>
It uses a global cache so that the query result is remembered if it is an actual canton.

In [22]:
def get_canton(univ, inst, cache):
    ''' Maps the input univ or inst to its canton.
        First, tries to query the APIs on the university.
          If no result was found, then query on the institution.
          If still no result is found, returns 'NA'.
        Caches the results in the cache dictionary.
    '''
    
    ################ HELPERS ################
    def get_location(query):
        ''' Returns a location in the form (lat,lng) for the given query word, or 'NA' if nothing was found.
            To do so, it uses the Google Places API.
        '''
        params = { 'key' : api_key, 'query' : query}
        places_json = fetch_json(places_api_url, params)
        
        if places_json == None:
            return 'NA'
        else:
            return places_json['geometry']['location']
    
    def get_canton_from_location(latlng):
        ''' Returns a canton from the provided location object, or 'NA' if nothing was found.
            To do so, it uses the Google Geocode API.
        '''
        params = {'latlng' : str(latlng['lat']) + ',' + str(latlng['lng']), 'sensor' : 'false'}
        geo_json = fetch_json(geocode_api_url, params)
        
        if geo_json == None:
            return 'NA'
        else:
            for r in geo_json['address_components']:
                if 'administrative_area_level_1' in r['types']:
                    return r['short_name']
            return 'NA'
    
    
    ################ FUNCTION CODE ################
    
    # use the cache for values that were previously retrieved
    if univ in cache:
        return cache[univ]
    elif inst in cache:
        return cache[inst]
    
    # determine the right query word (university or institution) and then perform a first lookup
    query  = get_query(univ, inst)
    latlng = get_location(query)
    
    # if it failed, try to lookup by institution name
    if latlng == 'NA' and univ not in ('NPO', 'FP', 'FINST'):
        query = inst
        latlng = get_location(query)
    
    # if that failed too, we have no further options
    if latlng == 'NA':
        return 'NA'
    
    # querying geocode api to get a canton from the location
    canton = get_canton_from_location(latlng)
    if canton != 'NA':
        cache[query] = canton
    return canton

### 2b - Using caching

To make good use of time and API requests, we use caching.<br>
To do so, we will group every entry by its university and try to find a canton for each entry. If we do, we register it in the cache:

In [23]:
cache = {}
univs = data.univ.groupby(data.univ)

for u in univs:
    canton = get_canton(u[0], None, {})
    if not canton == 'NA':
        cache[u[0]] = canton

As we can see, the canton for EPFL was successfully retrieved and cached. In subsequent calls for EPFL, the APIs will not be queried anymore.

In [24]:
cache['EPF Lausanne - EPFL']

'VD'

The initial cache will work for the following proportion of the universities. Then, it will grow as the queries start being executed on the dataframe entries.

In [25]:
print(len(cache), '/', len(univs))

60 / 76


### 2c - Retrieving the canton for each entry

We now iterate over all the entries of the dataframe, yielding exactly one `canton` value per entry.<br>
This should usually be done using a mapping function from one column to the other (as it was originally done), but so much can go wrong in 47,000+ entries (network failure, etc.), that we prefer the iterative solution. At least, we can save what has already been computed.

In [27]:
if force_download:
    cantons = pd.DataFrame(columns=('pnr', 'canton'))
    cantons = cantons.set_index('pnr')
    
    for r in data.itertuples(index=True):
        cantons.loc[r[0]] = get_canton(r.univ, r.inst, cache)
else:
    cantons = pd.read_csv('data/out_cantons.csv', sep=';')
    cantons = cantons.set_index('pnr')

<p>The proportion of retrieved `canton` values (vs. `NA` values) is:</p>

*(The requirement from the assignment was 95%)*

In [29]:
cantons = cantons.replace('NA', np.NaN)
proportion = (len(cantons.dropna()) / len(cantons)) * 100
print('%.2f' % proportion, '% of canton values are not null')

98.36 % of canton values are not null


We can, if the file isn't already saved, write the computed values to disk:

In [30]:
if force_download:
    cantons.to_csv('data/out_cantons.csv', sep=';', columns=['canton'])

### 2d - Putting everything back together

First, check that the two dataframes are of the same length:

In [31]:
len(data) == len(cantons)

True

Then add the `canton` column to the original data frame:

In [32]:
data_cantons = pd.concat([data, cantons], axis=1, join_axes=[data.index])
data_cantons.sample(5)

,univ,amount,title,inst,canton
pnr,,,,,
5724,Université de Genève - GE,322114.0,Etudes sur la régulation de la transcription c...,Département de Biologie Moléculaire Faculté de...,Genève
141953,Universität Zürich - ZH,5000.0,Perspectives on Wittgenstein's Philosophy of M...,Philosophisches Seminar Universität Zürich,ZH
53286,Paul Scherrer Institut - PSI,150000.0,R'EQUIP: Energieaufgelöste Reflexionselektrone...,Paul Scherrer Institut,AG
116563,Universität Zürich - ZH,72283.0,"Efficient, Diversified, and Secure Electricity...",Sozialökonomisches Institut (SOI) Universität ...,ZH
107476,ETH Zürich - ETHZ,67564.0,Micromechanics of magnetic shape-memory alloys,Institut für Angewandte Physik ETH Zürich,ZH


## Part 3 - Visualization

We need to get rid of the `NA` values for the `canton` field:

In [33]:
print(data_cantons.canton.isnull().sum(), 'rows were deleted')
data_cantons = data_cantons.dropna()

771 rows were deleted


### 3a - Why are you not Swiss??

We notice that some queries that were executed yielded a `canton` field that does not match an actual Swiss canton. It is for example the case for grants given to embassies in foreign countries. <br>
We need to filter the data to keep only the actual Swiss cantons. To that effect, we will get a list of `(short_name,long_name)` for each canton by using the provided `json` file. Then we can filter our dataframe.

In [34]:
import codecs

topo_file = codecs.open('data/ch-cantons.topojson.json', 'r', 'utf-8')
obj = json.load(topo_file)
topo_file.close()

In [35]:
canton_ids = {}
for c in obj['objects']['cantons']['geometries']:
    canton_ids[c['id']] = c['properties']['name']
canton_ids

{'AG': 'Aargau',
 'AI': 'Appenzell Innerrhoden',
 'AR': 'Appenzell Ausserrhoden',
 'BE': 'Bern/Berne',
 'BL': 'Basel-Landschaft',
 'BS': 'Basel-Stadt',
 'FR': 'Fribourg',
 'GE': 'Genève',
 'GL': 'Glarus',
 'GR': 'Graubünden/Grigioni',
 'JU': 'Jura',
 'LU': 'Luzern',
 'NE': 'Neuchâtel',
 'NW': 'Nidwalden',
 'OW': 'Obwalden',
 'SG': 'St. Gallen',
 'SH': 'Schaffhausen',
 'SO': 'Solothurn',
 'SZ': 'Schwyz',
 'TG': 'Thurgau',
 'TI': 'Ticino',
 'UR': 'Uri',
 'VD': 'Vaud',
 'VS': 'Valais/Wallis',
 'ZG': 'Zug',
 'ZH': 'Zürich'}

It looks like the data for Geneva was encoded by Google using the long name `Genève` instead of the code `GE`. We fix this:

In [36]:
data_cantons.canton = data_cantons.canton.replace('Genève', 'GE')

The number of entries whose `canton` field does not match an actual canton is:

In [37]:
not_swiss_predicate = [r for r in data_cantons.index if data_cantons.canton[r] not in canton_ids]
len(data_cantons.loc[not_swiss_predicate])

167

We use the defined predicate to drop the non-Swiss rows:

In [38]:
data_cantons = data_cantons.drop(not_swiss_predicate)

### 3b - Summing grants per canton

The first step is to group each entry by canton and sun all the grants allotted to each canton:

In [39]:
totals = data_cantons.groupby('canton').sum()

We have data for the following number of cantons:

In [40]:
print(len(totals), '/', '26')

23 / 26


For the sake of completeness, we add `0`s for the cantons that are not represented:

In [41]:
for c in canton_ids:
    if c not in totals.index:
        totals.loc[c] = 0

This gives the final list of cantons, sorted by descending total of grants received:

In [42]:
totals.sort_values(by='amount', ascending=False)

,amount
canton,
ZH,3.695896e+09
VD,2.421402e+09
GE,1.872504e+09
BE,1.563420e+09
BS,1.393876e+09
FR,4.533588e+08
NE,4.011148e+08
AG,1.272474e+08
TI,1.206732e+08


### 3c - Final map

<p>Let's split the grants totals into six slices.</p>

<p>First, we used a linear scale of 1/5th quantiles (20%, 40%, ...). However, we realized that in the top 26/5 = 5+ values, there was a very large gap between the first and the last value, but they were represented with the same color.<br>
The same goes for the last quantile: AR and GL were allotted grants but they were represented with the same color are NW, AI and OW, which had no money granted.</p>

<p>We then came up with the following quantiles scale for our representation:</p>

In [46]:
quantiles = totals.quantile([.10, .45, .60, .85, .90, 1.0])
scale = quantiles.amount.tolist()

This snippet will give us the final map, using our custom scale:

In [47]:
map_cantons = folium.Map(location=[46.9,8.3], zoom_start=8, tiles='cartodbpositron')
map_cantons.choropleth(geo_path = 'data/ch-cantons.topojson.json',
                       data = totals.reset_index(),
                       columns = ['canton', 'amount'],
                       key_on = 'feature.id',
                       threshold_scale = scale,
                       topojson = 'objects.cantons',
                       fill_color = 'GnBu',
                       legend_name = 'Total amount of SNF grants (CHF)'
)

In [50]:
map_cantons.save('map.html')
print('Total amount of SNF grants (CHF):')
map_cantons

Total amount of SNF grants (CHF):


N.B. For convenience, we include the following screenshot of the map. If the reader wants to see a dynamic map, they should download the `map.html` file that's included in the repo.<br>
<img src="screenshot-map.png">